In [1]:
import os
import re
import sys
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from Bio.PDB import *
from Bio.Data.IUPACData import protein_letters_1to3
from tqdm.notebook import tqdm
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [2]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.atomic import add_atomic_edges
from graphein.protein.graphs import construct_graph, read_pdb_to_dataframe
from graphein.protein.subgraphs import extract_subgraph_from_point
from graphein.protein.utils import save_graph_to_pdb
from graphein.molecule.edges.distance import compute_distmat, get_interacting_atoms
from graphein.protein.visualisation import plotly_protein_structure_graph

[03/13/24 00:14:52] WARNING  To use the Graphein submodule                                         ]8;id=289754;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=853591;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py#34\34]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             torch                                                                                 
                             To do so, use the following command: conda install -c pytorch torch                   

                    WARNING  To use the Graphein submodule                                         ]8;id=205141;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=4121;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py#45\45]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

[03/13/24 00:14:53] WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=248208;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=701029;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/visualisation.py#36\36]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=775939;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=823008;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/meshes.py#30\30]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [3]:
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [4]:
def extractResidueEnvironment(pdb_id, mutation, chain) -> pd.DataFrame:
    """
    Извлекает окружение мутирующего остатка, создавая подграф вокруг этого остатка.
    Подграф сохраняется в формате PDB и затем преобразуется в DataFrame для дальнейшего анализа.
    DataFrame скачивается в формате pickle

    - pdb_id (str): Идентификатор PDB структуры белка.
    - mutation (str): Мутация в формате 'AAnumAA', где AA - аминокислота, num - номер остатка.
    - chain (str): Цепь, к которой относится мутирующий остаток.

    Пример использования:
    >>> residue_environment = extractResidueEnvironment('1A2K', 'F149A', 'A')
    """

    aa, num_aa, mut_aa = re.match(r"([A-Za-z])(\d+)([A-Za-z])", mutation).groups() #F149A –> F+149+A
    aa = protein_letters_1to3[aa].upper() # F –> Phe –> PHE


    # Делаем граф из всего белка, если еще не делали на предыдущем шаге
    if pdb_id not in graphs_dict.keys():
        params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}
        config = ProteinGraphConfig(**params_to_change)
        graph = construct_graph(config=config, pdb_code=pdb_id)
        # Сохраняем граф в словаре
        graphs_dict[pdb_id] = graph
    else:
        # Если граф уже существует, используем его
        graph = graphs_dict[pdb_id]
    
    # Поиск координат СА мутирующего остатка
    for node, data in graph.nodes(data=True):
        if f'{chain}:{aa}:{num_aa}:CA' in node:
            mut_center = data['coords']
    

    # Выделение подграфа residue_environment_sg вокруг мутируемого остатка
    residue_environment_sg = extract_subgraph_from_point(graph, centre_point=(mut_center), radius=residue_environment_radius)


    # Сохранение подграфа residue_environment_sg в pdb формате
    save_graph_to_pdb(residue_environment_sg, f'./cutPDBs/{pdb_id}_{mutation}_cut.pdb')

    # Перевод pdb в датафрейм
    residue_environment = read_pdb_to_dataframe(f'./cutPDBs/{pdb_id}_{mutation}_cut.pdb')

    # Сохранение датафрейма в файл
    residue_environment.to_pickle(f'./res_env/{pdb_id}_{mutation}_env.pkl')

    return residue_environment

In [5]:
def extractEnvironmentFeats(pdb_id, mutation, residue_environment):
    
    # Создание меток фармакофоров для атомов окружения
    mol = Chem.MolFromPDBFile(f'./cutPDBs/{pdb_id}_{mutation}_cut.pdb')
    feats = factory.GetFeaturesForMol(mol)

    # Соберу названия фармакофоров и их координаты в один список
    feat_names_coords = []
    for feat in feats:
        feat_names_coords.append((feat.GetFamily(), tuple(feat.GetPos())))

    # Загружаю residue_environment, чтобы добавить колонку 'Pharmacophore'
    # residue_environment = pd.read_pickle(f'./res_env/{pdb_id}_{mutation}_env.pkl')
    residue_environment['Pharmacophore'] = ''

    # Функция для проверки совпадения всех трех координат и добавления имени
    def add_pharmacophore_name(row):
        current_coords = (row['x_coord'], row['y_coord'], row['z_coord'])
        for name, coords in feat_names_coords:
            if current_coords == coords:
                return name
        return ''

    # Применение функции к каждой строке DataFrame для заполнения колонки 'Pharmacophore'
    residue_environment['Pharmacophore'] = residue_environment.apply(add_pharmacophore_name, axis=1)

    # Удаление атомов, которые не фармакофоры
    res_env_pharm = residue_environment.loc[residue_environment['Pharmacophore'] != '']
    
    # Сохранение датафрейма в файл pkl
    # os.makedirs('./res_env_pharm', exist_ok=True)
    # res_env_pharm.to_pickle(f'./res_env_pharm/{pdb_id}_{mutation}_env.pkl')

    return res_env_pharm

In [6]:
# Проверка, как выглядит residue_environment_sg
# plotly_protein_structure_graph(residue_environment_sg, node_size_min=4, node_size_multiplier=2)

In [7]:
def calculateAtomicPairwiseDist(res_env_pharm) -> pd.DataFrame:

    coords = res_env_pharm.filter(like='_coord')
    distMatrix = compute_distmat(coords)

    pharmacophores = list(res_env_pharm['Pharmacophore'])
                          
    # делаю из массива distMatrix датафрейм с индексами означающими фармакофор
    distMatrix = pd.DataFrame(distMatrix, index=pharmacophores, 
                                        columns=pharmacophores)

    return distMatrix

In [8]:
def getFrequency(distMatrix, dist, classes):

    # Проверка наличия фармакофоров в окружении АК, если какого-то класса нет — возвращаем ноль
    if classes[0] not in distMatrix.index or classes[1] not in distMatrix.columns:
        return 0

    # Выбираем только значения на пересечении классов
    filt_distMatrix = distMatrix.loc[classes[0], classes[1]]
    
    # sum().sum() — первый раз по строкам, второй раз суммы строк
    # дополнительно проверяем > 0.001
    frequency = int(((filt_distMatrix > 0.001) & (filt_distMatrix < dist)).sum().sum())
    
    # если считаем для атомов одного класса, частоту делим пополам — dist(A1,A2) == dist(A2,А1)
    if classes[0] == classes[1]: 
        frequency /= 2 

    return frequency

In [9]:
def download_pdb(pdb_id):
    pdb_list = PDBList()
    pdb_list.retrieve_pdb_file(pdb_id, pdir="./PDBs", file_format="pdb")
    ent_filename = f"./PDBs/pdb{pdb_id}.ent"
    pdb_filename = f"./PDBs/{pdb_id}.pdb"
    os.rename(ent_filename, pdb_filename)

In [10]:
from Bio.PDB.DSSP import DSSP


def useDSSP(pdb_id, mutation, chain):
    p = PDBParser(QUIET=True)
    structure = p.get_structure('d', f'./PDBs/{pdb_id}.pdb')
    model = structure[0]
    dssp = DSSP(model, f'./PDBs/{pdb_id}.pdb', dssp='mkdssp', acc_array='Sander', file_type='PDB')

    aa, num_aa, mut_aa = re.match(r"([A-Za-z])(\d+)([A-Za-z])", mutation).groups()

    info_about_aa = dssp[(chain, (' ', int(num_aa), ' '))]
    secondary_structure = info_about_aa[2]
    rsa = info_about_aa[3]
    
    return rsa, secondary_structure

In [11]:
factory.GetFeatureFamilies()

('Donor',
 'Acceptor',
 'NegIonizable',
 'PosIonizable',
 'ZnBinder',
 'Aromatic',
 'Hydrophobe',
 'LumpedHydrophobe')

In [12]:
MutationSet = pd.read_csv('../datasets/mCSM-AB2_dataset_short.csv')

AtomClass = ('Donor', 
             'Acceptor', 
             'NegIonizable', 
             'PosIonizable', 
             'Hydrophobe')


residue_environment_radius = 10

# Будем считать кол-во контактов по сферическим слоям толщиной Dstep
Dstep = 4
Dmin = Dstep
Dmax = residue_environment_radius * 2 + 1  # Dmax включительно — +1

graphs_dict = {}
PDBs = []

In [13]:
def Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep):
    mCSM = []
    num_rows = len(MutationSet)

    with tqdm(total=num_rows, desc="Processing rows") as progressbar:  # чтобы следить за ходом выполнения
        for i, (index, Mutation) in enumerate(MutationSet.iterrows()):

            pdb_id = Mutation['PDB']
            mutation = Mutation['mutation']
            chain = Mutation['chain']
            ddG = Mutation['Exp. DDG']
            print(pdb_id, mutation)

            if pdb_id not in PDBs:
                download_pdb(pdb_id)
                PDBs.append(pdb_id)

            residue_environment = extractResidueEnvironment(pdb_id, mutation, chain)
            res_env_pharm = extractEnvironmentFeats(pdb_id, mutation, residue_environment)
            distMatrix = calculateAtomicPairwiseDist(res_env_pharm)
                
            
            mCSM_row = []
            for dist in range(Dmin, Dmax, Dstep):
                print(f'ОЧЕНЬ ВАЖНО ЭТО dist == {dist}')
                for classes in product(AtomClass, repeat=2): # берем комбинацию из 2 классов с повторениями
                    print(classes)
                    frequency = getFrequency(distMatrix, dist, classes)
                    print(frequency)
                    mCSM_row.append(frequency)
            
            rsa, secondary_structure = useDSSP(pdb_id, mutation, chain)
            mCSM_row.extend([rsa, secondary_structure]) # добавляю RSA и вторичную структуру
            mCSM_row.append(ddG) # добавляю экспериментальное ddG

            mCSM.append(mCSM_row) # добавляю полученную строку к матрице
            
            
            progressbar.update()

    return mCSM

In [ ]:
mCSM = Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep)

In [15]:
combinations = list(product(AtomClass, repeat=2))
mCSM_column_names = [f"{cls1}_{cls2}_{dist}" for dist in range(Dmin, Dmax, Dstep) for cls1, cls2 in combinations]
mCSM_column_names.extend(['RSA', 'Secondary_structure', 'ddG'])
mCSM = pd.DataFrame(mCSM, columns=mCSM_column_names)
mCSM

,Donor_Donor_4,Donor_Acceptor_4,Donor_NegIonizable_4,Donor_PosIonizable_4,Donor_Hydrophobe_4,Acceptor_Donor_4,Acceptor_Acceptor_4,Acceptor_NegIonizable_4,Acceptor_PosIonizable_4,Acceptor_Hydrophobe_4,...,PosIonizable_PosIonizable_20,PosIonizable_Hydrophobe_20,Hydrophobe_Donor_20,Hydrophobe_Acceptor_20,Hydrophobe_NegIonizable_20,Hydrophobe_PosIonizable_20,Hydrophobe_Hydrophobe_20,RSA,Secondary_structure,ddG
0,26.0,82,0,0,47,82,15.0,0,0,53,...,0,0,900,750,0,0,435.0,0.279188,H,-0.06
1,26.0,82,0,0,47,82,15.0,0,0,53,...,0,0,900,750,0,0,435.0,0.279188,H,0.00
2,29.0,89,0,0,59,89,17.0,0,0,59,...,0,0,1023,744,0,0,465.0,0.000000,H,-3.14
3,29.0,101,0,0,67,101,16.0,0,0,61,...,0,0,1295,1015,0,0,595.0,0.000000,H,-5.27


In [16]:
# # Попытка перенаправить вывод в файл для удобства анализа, неудачная

# with open('output.txt', 'w') as f:
#     # Сохраняем оригинальный стандартный вывод
#     original_stdout = sys.stdout
#     # Перенаправляем стандартный вывод в файл
#     sys.stdout = f
#     mCSM = Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep)

# sys.stdout = original_stdout

# # Вторая попытка, тоже неудачная, что ж
# %%capture output
# mCSM = Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep)

# with open('output_mCSM.txt', 'w') as f:
#     f.write(output.stderr)